In [ ]:
!pip install transformers sentencepiece textblob tqdm pandas

In [3]:
import pandas as pd
from textblob import TextBlob
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import torch
import os

INPUT_FILE = "/content/airbnb_paris_all_combined.csv"
OUTPUT_FILE = "airbnb_paris_translated_sentiment.csv"
SAVE_INTERVAL = 500
MAX_LEN = 512

df = pd.read_csv(INPUT_FILE)
if os.path.exists(OUTPUT_FILE):
    df_result = pd.read_csv(OUTPUT_FILE)
    print(f"🔁 偵測到已處理檔案，載入續跑...")
else:
    df_result = df.copy()
    df_result["translated_comment"] = ""
    df_result["sentiment_polarity"] = 0.0

model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

def is_obviously_english(text):
    if not isinstance(text, str) or not text.strip():
        return False
    english_clues = [" the ", " and ", " you ", " this ", " is ", " are ", " i ", " we ", " my ", " not "]
    return any(w in text.lower() for w in english_clues)

def translate_if_needed(text):
    if pd.isnull(text) or not isinstance(text, str) or not text.strip():
        return ""
    if is_obviously_english(text):
        return text[:MAX_LEN]
    try:
        inputs = tokenizer([text[:MAX_LEN]], return_tensors="pt", padding=True, truncation=True).to(model.device)
        translated = model.generate(**inputs)
        return tokenizer.decode(translated[0], skip_special_tokens=True)
    except:
        return "[Translation failed]"

def get_polarity(text):
    if not isinstance(text, str) or not text.strip():
        return 0.0
    return TextBlob(text).sentiment.polarity

untranslated_mask = df_result["translated_comment"].isnull() | (df_result["translated_comment"].str.strip() == "")
untranslated_indices = df_result[untranslated_mask].index.tolist()

print(f"📌 尚有 {len(untranslated_indices)} 筆資料尚未翻譯，開始處理...")

for idx in tqdm(untranslated_indices, desc="🔁 Translating & Analyzing"):
    comment = df_result.at[idx, "content"]

    translated = translate_if_needed(comment)
    polarity = get_polarity(translated)

    df_result.at[idx, "translated_comment"] = translated
    df_result.at[idx, "sentiment_polarity"] = polarity

    if (idx + 1) % SAVE_INTERVAL == 0 or (idx + 1) == untranslated_indices[-1]:
        df_result[["timestamp", "author", "content", "translated_comment", "sentiment_polarity"]].to_csv(
            OUTPUT_FILE, index=False, encoding="utf-8-sig"
        )
        print(f"💾 已儲存第 {idx + 1} 筆至 {OUTPUT_FILE}")

print("🎉 全部處理完畢，已儲存為 airbnb_paris_translated_sentiment.csv")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

...

💾 已儲存第 10000 筆至 airbnb_paris_translated_sentiment.csv



🔁 Translating & Analyzing:  96%|█████████▌| 10500/10935 [1:16:31<01:23,  5.24it/s]

💾 已儲存第 10500 筆至 airbnb_paris_translated_sentiment.csv



🔁 Translating & Analyzing: 100%|██████████| 10935/10935 [1:17:23<00:00,  2.35it/s]

💾 已儲存第 10934 筆至 airbnb_paris_translated_sentiment.csv
🎉 全部處理完畢，已儲存為 airbnb_paris_translated_sentiment.csv
